# Pre-trained NER model comparsion with GT (false positives and negatives)

Description:

- Apply a pre-trained NER model to your labeled text.
- Compare the model's output to your gold standard labels.
- Identify and classify errors:
1. False positives (entities identified by the model but not in the gold standard)
2. False negatives (entities missed by the model but present in the gold standard)
3. Incorrectly labeled entities
- Discuss possible reasons for these errors in the context of historical language.


In [1]:
import spacy
from spacy import displacy

# Load pre-trained model
nlp = spacy.load("en_core_web_sm")

# Sample medieval text (dummy)
medieval_text = """
In the year of our Lord 1066, William, Duke of Normandy, crossed the English Channel
with his great army. He landed at Pevensey and marched towards Hastings. There, he
met the forces of King Harold Godwinson on the field of battle. The Norman knights,
supported by archers from Brittany, defeated the Saxon fyrd and housecarls of Wessex.
With this victory, William claimed the throne of England and was crowned at
Westminster Abbey.
"""

# Apply the model
doc = nlp(medieval_text)

# Print model's output
print("Model's NER Output:")
for ent in doc.ents:
    print(f"{ent.text} - {ent.label_}")

# Our gold standard labels (from previous manual labeling)
gold_standard = [
    ("1066", "DATE"),
    ("William", "PERSON"),
    ("Duke of Normandy", "TITLE"),
    ("Normandy", "LOCATION"),
    ("English Channel", "LOCATION"),
    ("Pevensey", "LOCATION"),
    ("Hastings", "LOCATION"),
    ("King Harold Godwinson", "PERSON"),
    ("Norman", "NATIONALITY"),
    ("Brittany", "LOCATION"),
    ("Saxon", "NATIONALITY"),
    ("Wessex", "LOCATION"),
    ("England", "LOCATION"),
    ("Westminster Abbey", "LOCATION")
]

# Compare model output to gold standard
model_entities = set((ent.text, ent.label_) for ent in doc.ents)
gold_entities = set(gold_standard)

false_positives = model_entities - gold_entities
false_negatives = gold_entities - model_entities
correct = model_entities.intersection(gold_entities)

print("\nFalse Positives:")
for entity in false_positives:
    print(f"{entity[0]} - {entity[1]}")

print("\nFalse Negatives:")
for entity in false_negatives:
    print(f"{entity[0]} - {entity[1]}")

print("\nCorrectly Identified:")
for entity in correct:
    print(f"{entity[0]} - {entity[1]}")

# Identify incorrectly labeled entities
incorrectly_labeled = set()
for model_entity in model_entities:
    for gold_entity in gold_entities:
        if model_entity[0] == gold_entity[0] and model_entity[1] != gold_entity[1]:
            incorrectly_labeled.add((model_entity[0], model_entity[1], gold_entity[1]))

print("\nIncorrectly Labeled:")
for entity in incorrectly_labeled:
    print(f"{entity[0]} - Model: {entity[1]}, Gold: {entity[2]}")

# Visualize the model's output
displacy.render(doc, style="ent", jupyter=True)

Model's NER Output:
the year - DATE
William - PERSON
English - NORP
Pevensey - ORG
Hastings - WORK_OF_ART
Harold Godwinson - PERSON
Norman - PERSON
Brittany - GPE
Saxon - PERSON
Wessex - GPE
William - PERSON
England - GPE
Westminster Abbey - PERSON

False Positives:
Pevensey - ORG
England - GPE
Norman - PERSON
Hastings - WORK_OF_ART
the year - DATE
Harold Godwinson - PERSON
Saxon - PERSON
Brittany - GPE
Westminster Abbey - PERSON
Wessex - GPE
English - NORP

False Negatives:
English Channel - LOCATION
King Harold Godwinson - PERSON
Hastings - LOCATION
Brittany - LOCATION
Duke of Normandy - TITLE
Normandy - LOCATION
Wessex - LOCATION
Pevensey - LOCATION
Norman - NATIONALITY
1066 - DATE
England - LOCATION
Saxon - NATIONALITY
Westminster Abbey - LOCATION

Correctly Identified:
William - PERSON

Incorrectly Labeled:
Norman - Model: PERSON, Gold: NATIONALITY
Hastings - Model: WORK_OF_ART, Gold: LOCATION
Saxon - Model: PERSON, Gold: NATIONALITY
England - Model: GPE, Gold: LOCATION
Brittany -

## Solution

False Positives:

Modern NER models might identify entities that weren't considered named entities in medieval contexts. For example, common nouns that have become proper nouns in modern usage.
The model might over-identify locations or organizations due to capitalization patterns that differ from modern usage.


False Negatives:

Historical titles or roles (e.g., "Duke of Normandy") might not be recognized as entities by models trained on modern text.
Place names that are no longer in common use (e.g., "Wessex") might be missed.
Complex historical names (e.g., "King Harold Godwinson") might not be fully captured.


Incorrectly Labeled Entities:

The model might misclassify nationalities (e.g., "Norman", "Saxon") as locations or organizations due to their usage in modern contexts.
Historical locations might be misclassified as organizations or vice versa (e.g., "Westminster Abbey" could be seen as either).


General Challenges:

Spelling variations: Medieval texts often have inconsistent spellings, which can confuse models trained on modern standardized spelling.
Context-dependent entities: Some words might be entities in certain contexts but not in others, and historical contexts can be particularly challenging.
Archaic language: The use of archaic terms or sentence structures can impact the model's ability to correctly identify and classify entities.
Limited training data: Pre-trained models often lack sufficient examples of medieval or historical text, leading to lower performance on these texts.



To improve performance for medieval texts, you might consider:

Fine-tuning the model on a corpus of annotated medieval texts.
Creating custom entity types that better reflect medieval concepts and roles.
Implementing rule-based systems to complement the statistical model, especially for known historical entities.
Using ensemble methods that combine multiple models or approaches to leverage their respective strengths.